In [41]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#import pandas as pd
#import numpy as np

from nereus.nereus_scraper import TideScraper
from oauth_setup import oauth_login

t = oauth_login()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
scraper = TideScraper()

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [51]:
print(scraper.get_tides_info())

San Francisco Bay Entrance (Golden Gate), California  [2018-04-03]:
Sun rises at 07:45, sets at 20:27
Ebb 04:34-11:24 (Max: -4.3 @ 07:34)
Flood 11:24-17:37 (Max: 3.0 @ 14:19)
Ebb 17:37-23:30 (Max: -2.8 @ 20:02)



In [52]:
tmp = t.statuses.update(status=scraper.get_tides_info())

In [34]:
tweetlist = ['Uno...', '...due...', 'Tre.. Prova.']

for line in tweetlist: 
    t.statuses.update(status=line)
    print(line, '...')
    time.sleep(15) # Sleep for 15 seconds

print("All done!")

Uno... ...
...due... ...
Tre.. Prova. ...
All done!


In [ ]:
print(format_tides_info(read_tides()))

In [ ]:
'http://tides.mobilegeographics.com/zones/'
'http://tides.mobilegeographics.com/zones/:Europe/'
'http://tides.mobilegeographics.com/zones/:Europe/Gibraltar'

'http://tides.mobilegeographics.com/index.html'